In [1]:
from extract_contacts_and_attentions import *

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [2]:
# base_dir = '/Users/williamharrigan/Desktop/Github/contact_site_classifier/attention_classifier/data_files/'
# desktop = '/Users/williamharrigan/Desktop/'
# structure_dir = base_dir +'structure_files/'
# fasta_dir = base_dir +'fasta_files/'
# casp_dir = '/Users/williamharrigan/Desktop/UH/Year_2/Research/contact_site_classifier/casp7/' 
# casp_95 = casp_dir + 'training_95'

In [17]:
base_dir = '/home/wlh/contact_prediction/'
structure_dir = base_dir +'structure_files/'
fasta_dir = base_dir +'fasta_file/'
casp_95 = base_dir + 'training_95'

In [3]:
warnings.simplefilter('ignore', PDBConstructionWarning)
parser = PDBParser()  

In [5]:
# Get sequences from CASP7 file
prot_data_dict = parse_casp7_file(casp_95)

protein_id_counts = Counter(protein_id.split('_')[0] for protein_id in prot_data_dict)
single_occurence_ids = [protein_id for protein_id, count in protein_id_counts.items() if count == 1]

# protein_data = generate_fastas(single_occurence_ids)
protein_data = load_fastas(fasta_dir)
same_sequence_ids = check_casp_pdb_seqs(protein_data)
in_contact_sites, non_contact_sites, subset_non_contact_sites = contacts_per_pdb(same_sequence_ids,protein_data)
contact_data = generate_contact_data(in_contact_sites, subset_non_contact_sites)


In [18]:
seed_value = 67
random.seed(seed_value)
n_sequences = 300

sequence_ids = random.sample(same_sequence_ids, n_sequences)
# sequence_ids = same_sequence_ids
print(sequence_ids[:5])

['1BUE', '1FK5', '1QK9', '1PPQ', '1T5Q']


In [30]:
if '4MT2' in same_sequence_ids:
    print('yes')

## Model

In [14]:
from sklearn.svm import LinearSVC


In [19]:
X, y = output_x_y(sequence_ids, contact_data, protein_data)

Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Itera

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.4, random_state=55)

In [27]:
# linear_svc = LinearSVC(random_state=55)
linear_svc = LinearSVC(C=10, class_weight='balanced', dual=True, loss='hinge')
linear_svc.fit(X_train, y_train)

/Users/williamharrigan/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=100, class_weight='balanced', dual=True, loss='hinge')

In [28]:
y_pred = linear_svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.88


In [29]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.8827895469305705
Precision: 0.9144189144189144
Recall: 0.8457037265167847
